In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np 
import os
import gc
import subprocess
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from PIL import Image
from skimage import data, io, filters, color
from skimage.transform import rescale, resize, downscale_local_mean 
from skimage.measure import block_reduce
from imblearn.over_sampling import SMOTE
import tensorflow_addons as tfa

In [2]:
tf.__version__

'2.7.0'

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [8]:
'''Creates a numpy array file for the data with RGB tuples for each pixel. 
This method is used for Neural Networks where tuples are accepted
as possible inputs.

Parameters:
dimension - an integer representing desired of the image:
            dimension x dimension

save - a boolean representing whether the arrays should be saved or not
Returns:
x - an array of all of the data with features of desired dimension
y - an array of the outputs


'''
def dataForNN(dimension, save):
    cwd = os.getcwd()
    metadata = pd.read_csv(cwd+'/data/HAM10000_metadata.csv')
    directory1 = cwd+'/data/HAM10000_images_part_1/'
    directory2 = cwd+'/data/HAM10000_images_part_2/'
    i=0
    y=np.empty((10015,7))
    x = np.empty((10015, dimension,dimension,3))
    c_type = {"akiec":np.array([1,0,0,0,0,0,0]),
         "bcc":np.array([0,1,0,0,0,0,0]),
         "bkl":np.array([0,0,1,0,0,0,0]),
         "df":np.array([0,0,0,1,0,0,0]),
         "mel":np.array([0,0,0,0,1,0,0]),
         "nv":np.array([0,0,0,0,0,1,0]),
         "vasc":np.array([0,0,0,0,0,0,1])}
    i=0
    for imagename in os.listdir(directory1):
        x[i] = resize(mpimg.imread(directory1+imagename), 
                    output_shape=(dimension,dimension,3), 
                    preserve_range = True,anti_aliasing=True).astype(int)
        y[i] = c_type[metadata.loc[metadata['image_id'] == 
                    imagename[0:len(imagename)-4]]['dx'].array[0]]
        i+=1
    for imagename in os.listdir(directory2):
        x[i] = resize(mpimg.imread(directory2+imagename),
                output_shape=(dimension,dimension,3), preserve_range = True,
                anti_aliasing=True).astype(int)
        y[i] = c_type[metadata.loc[metadata['image_id'] == 
                    imagename[0:len(imagename)-4]]['dx'].array[0]]
        i+=1
    #Will be saved outside any folder
    if(save):
        np.save(str(dimension)+'_'+str(dimension)+"_Vector", x)
        np.save('Outputs_Vector', y)
    return x, y

In [9]:
x,y = dataForNN(224,True)
#Will be saved to use in this format as well
#x = np.load("224_224_Vector.npy")
#y = np.load("Outputs_Vector.npy")

In [10]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

In [11]:
#Mobile net requires range of -1 to 1
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size = 0.2)

In [13]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

In [14]:
base_model.trainable = False
prediction_layer = tf.keras.layers.Dense(7, activation='softmax')
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [79]:
inputs = tf.keras.Input(shape=(224, 224, 3))
p = preprocess_input(inputs)
p = base_model(p, training=False)
p = global_average_layer(p)
p = tf.keras.layers.Dropout(0.2)(p)
outputs = prediction_layer(p)
model = tf.keras.Model(inputs, outputs)

In [80]:
base_learning_rate = 1e-3
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [81]:
history = model.fit(x_train, y_train, batch_size=64, epochs=5, validation_data=(x_val,y_val), validation_batch_size=128, workers=4)

Epoch 1/5
101/101 [==============================] - 35s 340ms/step - loss: 1.0372 - accuracy: 0.6672 - val_loss: 0.8027 - val_accuracy: 0.7112
Epoch 2/5
101/101 [==============================] - 32s 319ms/step - loss: 0.7748 - accuracy: 0.7274 - val_loss: 0.7157 - val_accuracy: 0.7424
Epoch 3/5
101/101 [==============================] - 33s 328ms/step - loss: 0.7031 - accuracy: 0.7524 - val_loss: 0.6904 - val_accuracy: 0.7492
Epoch 4/5
101/101 [==============================] - 33s 328ms/step - loss: 0.6585 - accuracy: 0.7660 - val_loss: 0.6913 - val_accuracy: 0.7530
Epoch 5/5
101/101 [==============================] - 33s 324ms/step - loss: 0.6290 - accuracy: 0.7738 - val_loss: 0.6582 - val_accuracy: 0.7598


In [82]:
model.evaluate(x_test,  y_test, verbose=2)

63/63 - 9s - loss: 0.7070 - accuracy: 0.7424 - 9s/epoch - 140ms/step


[0.7070242166519165, 0.7423864006996155]

In [83]:
base_model.trainable = True
# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [84]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [88]:
len(model.trainable_variables)

56

In [86]:
fine_tune_epochs = 5
total_epochs =  5 + fine_tune_epochs

history_fine = model.fit(x_train,y_train,batch_size=64,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=(x_val,y_val), validation_batch_size=128)

Epoch 5/10
101/101 [==============================] - 49s 475ms/step - loss: 0.8440 - accuracy: 0.7081 - val_loss: 0.6953 - val_accuracy: 0.7467
Epoch 6/10
101/101 [==============================] - 46s 454ms/step - loss: 0.6242 - accuracy: 0.7706 - val_loss: 0.5982 - val_accuracy: 0.7842
Epoch 7/10
101/101 [==============================] - 46s 452ms/step - loss: 0.5283 - accuracy: 0.8087 - val_loss: 0.5658 - val_accuracy: 0.7941
Epoch 8/10
101/101 [==============================] - 46s 456ms/step - loss: 0.4475 - accuracy: 0.8377 - val_loss: 0.5548 - val_accuracy: 0.8091
Epoch 9/10
101/101 [==============================] - 46s 461ms/step - loss: 0.3565 - accuracy: 0.8727 - val_loss: 0.6147 - val_accuracy: 0.8047
Epoch 10/10
101/101 [==============================] - 47s 464ms/step - loss: 0.2809 - accuracy: 0.8990 - val_loss: 0.5998 - val_accuracy: 0.7742


In [87]:
model.evaluate(x_test,  y_test, verbose=2)

63/63 - 9s - loss: 0.6618 - accuracy: 0.7494 - 9s/epoch - 137ms/step


[0.6618484258651733, 0.7493759393692017]

In [7]:
base_model2 = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')
base_model2.trainable=False
prediction_layer = tf.keras.layers.Dense(7, activation='softmax')
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [8]:
inputs = tf.keras.Input(shape=(224, 224, 3))
p = preprocess_input(inputs)
p = base_model2(p, training=False)
p = global_average_layer(p)
p = tf.keras.layers.Dropout(0.2)(p)
outputs = prediction_layer(p)
model2 = tf.keras.Model(inputs, outputs)

In [9]:
base_learning_rate = 1e-3
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tfa.losses.SigmoidFocalCrossEntropy(),
              metrics=['accuracy'])

In [12]:
history2 = model2.fit(x_train, y_train, batch_size=64, epochs=5, validation_data=(x_val,y_val), validation_batch_size=128, workers=4)

2022-05-13 12:49:33.507146: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/5
101/101 [==============================] - 37s 347ms/step - loss: 0.2466 - accuracy: 0.6216 - val_loss: 0.1791 - val_accuracy: 0.7043
Epoch 2/5
101/101 [==============================] - 34s 334ms/step - loss: 0.1741 - accuracy: 0.7017 - val_loss: 0.1577 - val_accuracy: 0.7143
Epoch 3/5
101/101 [==============================] - 33s 322ms/step - loss: 0.1506 - accuracy: 0.7276 - val_loss: 0.1544 - val_accuracy: 0.7187
Epoch 4/5
101/101 [==============================] - 33s 329ms/step - loss: 0.1401 - accuracy: 0.7443 - val_loss: 0.1566 - val_accuracy: 0.7068
Epoch 5/5
101/101 [==============================] - 33s 324ms/step - loss: 0.1279 - accuracy: 0.7580 - val_loss: 0.1399 - val_accuracy: 0.7374


In [13]:
base_model2.trainable = True
# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model2.layers[:fine_tune_at]:
  layer.trainable = False

In [14]:
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
              loss=tfa.losses.SigmoidFocalCrossEntropy(),
              metrics=['accuracy'])

In [15]:
fine_tune_epochs = 5
total_epochs =  5 + fine_tune_epochs

history2_fine = model2.fit(x_train,y_train,batch_size=64,
                         epochs=total_epochs,
                         initial_epoch=history2.epoch[-1],
                         validation_data=(x_val,y_val), validation_batch_size=128)

Epoch 5/10
101/101 [==============================] - 51s 490ms/step - loss: 0.1633 - accuracy: 0.7006 - val_loss: 0.1370 - val_accuracy: 0.7399
Epoch 6/10
101/101 [==============================] - 48s 479ms/step - loss: 0.1248 - accuracy: 0.7638 - val_loss: 0.1275 - val_accuracy: 0.7561
Epoch 7/10
101/101 [==============================] - 48s 481ms/step - loss: 0.1054 - accuracy: 0.8046 - val_loss: 0.1218 - val_accuracy: 0.7686
Epoch 8/10
101/101 [==============================] - 48s 472ms/step - loss: 0.0906 - accuracy: 0.8318 - val_loss: 0.1317 - val_accuracy: 0.7467
Epoch 9/10
101/101 [==============================] - 48s 472ms/step - loss: 0.0778 - accuracy: 0.8510 - val_loss: 0.1308 - val_accuracy: 0.7860
Epoch 10/10
101/101 [==============================] - 47s 466ms/step - loss: 0.0682 - accuracy: 0.8664 - val_loss: 0.1272 - val_accuracy: 0.7773


In [99]:
fine_tune2_epochs = 5
total2_epochs=5+total_epochs
history2_fine2 = model2.fit(x_train,y_train,batch_size=64,
                         epochs=total2_epochs,
                         initial_epoch=history2_fine.epoch[-1],
                         validation_data=(x_val,y_val), validation_batch_size=128)

Epoch 10/15
101/101 [==============================] - 49s 478ms/step - loss: 0.0670 - accuracy: 0.8728 - val_loss: 0.1059 - val_accuracy: 0.8147
Epoch 11/15
101/101 [==============================] - 46s 452ms/step - loss: 0.0581 - accuracy: 0.8911 - val_loss: 0.1002 - val_accuracy: 0.8172
Epoch 12/15
101/101 [==============================] - 46s 451ms/step - loss: 0.0533 - accuracy: 0.8981 - val_loss: 0.1210 - val_accuracy: 0.8203
Epoch 13/15
101/101 [==============================] - 45s 449ms/step - loss: 0.0390 - accuracy: 0.9271 - val_loss: 0.1094 - val_accuracy: 0.8322
Epoch 14/15
101/101 [==============================] - 46s 458ms/step - loss: 0.0349 - accuracy: 0.9396 - val_loss: 0.1158 - val_accuracy: 0.8334
Epoch 15/15
101/101 [==============================] - 45s 446ms/step - loss: 0.0268 - accuracy: 0.9552 - val_loss: 0.2451 - val_accuracy: 0.7948


In [16]:
y_pred = model2.evaluate(x_test, y_test)

63/63 [==============================] - 9s 144ms/step - loss: 0.1159 - accuracy: 0.7793
